# Global Natural NPP Production


In [12]:
import datetime

import ee
from notebook_setup import *

from eii.compute.npp import setup_predictor_stack
from eii.utils import create_assets_folder

In [ ]:
POSTFIX = ""
MODEL_PATH_MEAN = f"{MODEL_ASSETS_PATH}/npp_rf_model_mean_v1{POSTFIX}"
MODEL_PATH_STD = f"{MODEL_ASSETS_PATH}/npp_rf_model_std_v1{POSTFIX}"

APPLY_LAND_MASK = True
RESOLUTION = 300

# Output asset path (single image)
OUTPUT_ASSET_PATH = f"{NPP_PREDICTION_ASSET_PATH}/natural_npp_v1"

# Optional AOI (set to None for global export)
AOI = None

# Global bounds (exclude polar regions with sparse data)
GLOBAL_BOUNDS = ee.Geometry.Rectangle([-180, -60, 180, 75], proj="EPSG:4326", geodesic=False)

# Brazil test tile (same size as Munich: 2° x 1.5°)
BRAZIL_BOUNDS = ee.Geometry.Rectangle([-55.0, -11.0, -53.0, -9.5], proj="EPSG:4326", geodesic=False)
# AOI = BRAZIL_BOUNDS

EXPORT_REGION = AOI if AOI is not None else GLOBAL_BOUNDS

print(f"Model Path (Mean): {MODEL_PATH_MEAN}")
print(f"Model Path (Std): {MODEL_PATH_STD}")
print(f"Output Asset Path: {OUTPUT_ASSET_PATH}")
print(f"Resolution: {RESOLUTION}m")
print("Export region: AOI" if AOI is not None else "Export region: Global bounds")

Model Path (Mean): projects/landler-open-data/assets/eii/intermediate/functional/models/npp_rf_model_mean_v1_climate_aridity_soil_location_chelsa_hyper
Model Path (Std): projects/landler-open-data/assets/eii/intermediate/functional/models/npp_rf_model_std_v1_climate_aridity_soil_location_chelsa_hyper
Output Asset Path: projects/landler-open-data/assets/eii/intermediate/functional/predictions/natural_npp_v4
Resolution: 300m
Export region: Global bounds


In [ ]:
model_mean = ee.Classifier.load(MODEL_PATH_MEAN)
model_std = ee.Classifier.load(MODEL_PATH_STD)

print(f"Loaded mean NPP model from: {MODEL_PATH_MEAN}")
print(f"Loaded std NPP model from: {MODEL_PATH_STD}")

--- Load Trained Models ---
Loaded mean NPP model from: projects/landler-open-data/assets/eii/intermediate/functional/models/npp_rf_model_mean_v1_climate_aridity_soil_location_chelsa_hyper
Loaded std NPP model from: projects/landler-open-data/assets/eii/intermediate/functional/models/npp_rf_model_std_v1_climate_aridity_soil_location_chelsa_hyper


In [ ]:
def get_land_mask(resolution: int = 300) -> ee.Image:
    """
    Create a land mask from MODIS land cover.

    Returns a binary mask where 1 = land, 0 = water/ocean.
    Using MODIS LC_Type1 where class 17 = water bodies.
    """
    modis_lc = ee.ImageCollection("MODIS/061/MCD12Q1").first()
    land_mask = modis_lc.select("LC_Type1").neq(17).rename("land_mask")
    return land_mask.reproject(crs="EPSG:4326", scale=resolution)


predictors = setup_predictor_stack(resolution=RESOLUTION).clip(EXPORT_REGION)

if APPLY_LAND_MASK:
    predictors = predictors.updateMask(get_land_mask(RESOLUTION))

predicted_npp_mean = predictors.classify(model_mean).rename("natural_npp_mean").toFloat()
predicted_npp_std = predictors.classify(model_std).rename("natural_npp_std").toFloat()

npp_oneshot = ee.Image.cat([predicted_npp_mean, predicted_npp_std]).set(
    {
        "model_version": GEE_MODEL_VERSION,
        "resolution": RESOLUTION,
        "prediction_date": datetime.datetime.now().strftime("%Y-%m-%d"),
        "model_path_mean": MODEL_PATH_MEAN,
        "model_path_std": MODEL_PATH_STD,
        "land_mask_applied": APPLY_LAND_MASK,
        "export_region": "aoi" if AOI is not None else "global_bounds",
    }
)

print("✓ Built natural NPP image")
print(f"  Bands: {npp_oneshot.bandNames().getInfo()}")

✓ Built natural NPP image
  Bands: ['natural_npp_mean', 'natural_npp_std']


In [ ]:
create_assets_folder("/".join(OUTPUT_ASSET_PATH.split("/")[:-1]))

try:
    ee.data.getAsset(OUTPUT_ASSET_PATH)
    print(f"Asset already exists: {OUTPUT_ASSET_PATH}")
    RUN_EXPORT = False
except ee.EEException:
    RUN_EXPORT = True

if RUN_EXPORT:
    description = f"NaturalNPP_OneShot_{POSTFIX}"
    task = ee.batch.Export.image.toAsset(
        image=npp_oneshot,
        description=description,
        assetId=OUTPUT_ASSET_PATH,
        region=EXPORT_REGION,
        scale=RESOLUTION,
        maxPixels=1e13,
        crs="EPSG:4326",
        pyramidingPolicy={".default": "mean"},
    )
    task.start()

    print(f"Exporting to: {OUTPUT_ASSET_PATH}")
    print(f"Region: {EXPORT_REGION.getInfo()['coordinates']}")

Exporting to: projects/landler-open-data/assets/eii/intermediate/functional/predictions/natural_npp_v4
Description: NaturalNPP_OneShot_climate_aridity_soil_location_chelsa_hyper
Region: [[[-180, -60], [180, -60], [180, 75], [-180, 75], [-180, -60]]]
Scale: 300m
NOTE: Single export task; GEE handles internal tiling.
✓ Export task started: X4GNSLWBFTSVZ34HUNNJ7YCD
Monitor at: https://code.earthengine.google.com/tasks


In [22]:
task.status()

{'state': 'RUNNING',
 'description': 'NaturalNPP_OneShot_climate_aridity_soil_location_chelsa_hyper',
 'priority': 100,
 'creation_timestamp_ms': 1768504169490,
 'update_timestamp_ms': 1768504517505,
 'start_timestamp_ms': 1768504176014,
 'task_type': 'EXPORT_IMAGE',
 'attempt': 1,
 'id': 'X4GNSLWBFTSVZ34HUNNJ7YCD',
 'name': 'projects/landler-open-data/operations/X4GNSLWBFTSVZ34HUNNJ7YCD'}